In [1]:
# from flask_other.app import app
from flask import Flask, render_template, session, request
from flask_socketio import SocketIO, emit
import os,sys
# sys.path.append('/home/kai/data/wei/Chatbot1.0/')
sys.path.append('../')
# sys.path.append('/home/kai/data/Chatbot1.0/API/v1/frontend/')
app=Flask("demo")
from flask import  render_template,request
from app.mong_database import MongoManager
import time
import re
import uuid
mongo_db=MongoManager(server_ip='chatbotdb')

import urllib.request
from urllib.parse import quote
import pandas as pd
import numpy as np
import time
import re
import requests

import asyncio

##################################
async_mode = None
url = 'http://localhost:8889/chatbotv1'

app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret!'
socketio = SocketIO(app, async_mode=async_mode)


def get_data(sentence):
    
    response = requests.get(url, timeout=10, params={'data':sentence, 
                                                     'action':'chat', 
                                                     'sessionId':cache.sessionId})
    print(response)
    
    if response.status_code == 200:
        req = response.json()
        if req.get('status').lower() == 'successful':
            msg = req.get('message')
        else:
            msg = req

        return msg
    else:
        return 'Internet Connection Issue, error code {}'.format(response.status_code)
def get_data(sentence):
    
    return 'data get'

async def hello(websocket, path):
    t = 2
    if t == 2:
        test = "test"
        websocket.send(test)
        t = 3
    while 1:
        name = await websocket.recv()
        print(f"< {name}")

        greeting = f"Hello {name}!"

        await websocket.send(greeting)
        print(f"> {greeting}")




@app.route('/')
@app.route('/index')
def index0():
    return render_template(
                           # "index0.html",
                           "index.html",
                           )
@app.route("/predict", methods= ["POST"])
def background_process():
    if request.method == 'POST':
        try:
            
            
            query = request.form.get('query')#前端查询的内容
            u_id = request.form.get('uid')
            print(u_id)
            if query:

                    print('query',query)
                    # time.sleep(5)
                    result = get_data(query)
                    print('result',result)
                    mongo_db.save_query(query, str(result))
                    print('saving to mongo db successfully!')
                    return str(result)
                    

            else:

                    return str('请输入查询内容')


        except Exception as e:
            print(e)

            if 'duplicate' in str(e):
                e_str = e.details['errmsg']
                dup_id=re.search('\{ : "(.*)" \}',e_str).group(1)
                print('重复查询同一句话,存储时使用相同的_id_',dup_id)
                mongo_db.update_dup_query( dup_id, str(result))
                return str(result)

            else:
                print(e)
                print('有问题，MM出故障啦')
                return str('MM出故障啦')

        # finally:
        #     # print(e)
        #     print('有问题，MM出故障啦。。')
        #     return str('MM出故障啦。。')

    else:
        return 'ok'
    
@app.route("/test2", methods = ['POST'])
def ttest():
    if request.method == 'POST':
        
        cookieid = request.form.get('cookie');
        if cookieid:
            print('already exists')
            print(cookieid)
            return('')
        else:
            print('new user')
            uid = str(uuid.uuid())
            return(uid)
    
    
@app.route("/newConversation", methods= ["POST"])
def new_conversation():
    
    #cache.new_conversation()
    if request.method == 'POST':
        cookie = request.form.get('uid');
        if cookie:
            print(cookie)
            return cookie
        else:
            print('cookie lost')
            return('error')

@socketio.on('connect')
def connect():
    print('connect')
class Cache:
    def __init__(self):
        self.new_conversation()
            
    def new_conversation(self):
        req = requests.get(url, timeout=10, params={'action':'create'})
        if req.status_code == 200:
            msg = req.json()
            if msg['status'] == 'successful':
                self.sessionId = msg['message']['sessionId']
                
                print(self.sessionId)
            else:
                
                print(req.content)
                raise ValueError('cannot get new sessionId, cannot start chat')
        else:
            
            print(req.content)       
# cache = Cache()   




chatbotdb


In [ ]:

app.debug = False
port = 6006
print('running at http://10.0.24.31:{}'.format(port))

# app.run(host='0.0.0.0',port=port)
socketio.run(app,'0.0.0.0', port)

#这样用来监听所有的ip，团队调试用

running at http://10.0.24.31:6006
